In [ ]:
import random
import time 
import json 

# ----------------------------------------------------
# Herramientas Funcionales
# ----------------------------------------------------
def clasificacion(query):
    # Herramienta de Razonamiento
    if "reprogramar" in query or "cambiar hora" in query:
        return {"categoria": "reprogramacion_cita"}
    elif "horario" in query or "visita" in query:
        return {"categoria": "pregunta_frecuente"}
    elif "reclamo" in query:
        return {"categoria": "reclamo"}
    return {"categoria": "otra_consulta"}

def contexto(query, knowledge_base):
    # Herramienta de Consulta (Simulación RAG)
    if "visita" in query or "horario" in query:
        return knowledge_base["horarios_visita"]
    elif "medicamento" in query:
        return knowledge_base["farmacia_disponibilidad"]
    elif "dolor" in query or "urgencias" in query:
        return knowledge_base["urgencias"]
    return None

def generador_respuesta(query, context, memory_buffer):
    # Herramienta de Escritura
    
    # Coherencia de memoria
    is_in_reclamo_flow = any("reclamo" in entry for entry in memory_buffer[-2:])
    if is_in_reclamo_flow and "problema" in query:
        return "Entendido. Su problema ha sido registrado como parte del reclamo escalado. ¿Necesita algo mas?"
    
    # Uso del Contexto RAG
    if context and "horario" in context:
        return "El horario de visita es de 15:00 a 17:00 de lunes a viernes."
    elif context and "farmacia" in context:
        return "Puede consultar la disponibilidad de su medicamento en el portal web del hospital, ya que el inventario se actualiza constantemente."
    elif context and "urgencias" in context:
        return "Nuestros protocolos de urgencia sugieren que ante un dolor intenso es mejor dirigirse a la sala de emergencias. Le sugiero ir a la brevedad."
    
    return "Disculpe, no tengo una respuesta precisa para su consulta. Por favor, reformule su pregunta."

# ----------------------------------------------------
# Clase Principal del Agente
# ----------------------------------------------------
class HospitalAIAgent:
    def __init__(self):
        # Memoria de Largo Plazo (Base de Conocimiento)
        self.knowledge_base = {
            "urgencias": "Nuestros protocolos de urgencia indican que ante un dolor intenso...",
            "horarios_visita": "El horario de visita es de 15:00 a 17:00 de lunes a viernes.",
            "farmacia_disponibilidad": "La farmacia central tiene un inventario actualizado. La disponibilidad de farmacos se puede consultar en el portal web...",
            "protocolo_reclamos": "Para registrar un reclamo, necesitamos la fecha del incidente y el servicio afectado."
        }
        # Memoria de Corto Plazo
        self.memory_buffer = []

        # Métricas de Observabilidad
        self.metrics = {
            "total_calls": 0,
            "successful_automatic_responses": 0, 
            "total_latency_ms": 0,
            "total_derivations": 0, 
            "simulated_classification_errors": 0 
        }

    def update_memory(self, user_query, agent_response):
        self.memory_buffer.append(f"Usuario: {user_query}")
        self.memory_buffer.append(f"Agente: {agent_response}")
        self.memory_buffer = self.memory_buffer[-6:] 
        
    def simulate_processing_delay(self):
        # Simula el tiempo de respuesta del LLM/RAG
        delay = random.uniform(0.3, 1.5)
        time.sleep(delay)
        return int(delay * 1000)

    def calculate_metrics(self):
        # Cálculo de métricas
        total_calls = self.metrics["total_calls"]
        if total_calls == 0:
            return {}
            
        avg_latency = self.metrics["total_latency_ms"] / total_calls
        precision_auto = (self.metrics["successful_automatic_responses"] / total_calls) * 100
        derivation_rate = (self.metrics["total_derivations"] / total_calls) * 100
        error_rate = (self.metrics["simulated_classification_errors"] / total_calls) * 100
        
        return {
            "avg_latency_ms": f"{avg_latency:.2f}",
            "precision_auto_pct": f"{precision_auto:.2f}%",
            "derivation_rate_pct": f"{derivation_rate:.2f}%",
            "error_rate_pct": f"{error_rate:.2f}%"
        }

    # Orquestación, Planificación y Trazabilidad
    def process_query(self, user_query):
        start_time = time.time()
        self.metrics["total_calls"] += 1
        
        # 1. Razonamiento: Clasificar
        classification = clasificacion(user_query)
        category = classification.get("categoria", "otra_consulta")
        derivation_needed = False
        
        # Simulación de Error de Clasificación
        if random.random() < 0.1:
            category = "otra_consulta" 
            self.metrics["simulated_classification_errors"] += 1

        # 2. Consulta: Recuperación de Contexto (RAG)
        context_data = contexto(user_query, self.knowledge_base)
        
        # 3. Planificación y Toma de Decisiones
        
        if category == "reclamo":
            # Derivación Obligatoria
            final_response = "Para registrar formalmente su reclamo, necesito sus datos de contacto y la fecha del incidente. Un funcionario recibirá la informacion directamente."
            derivation_needed = True 
            self.metrics["total_derivations"] += 1 
        
        elif category == "reprogramacion_cita":
            # Flujo automatizado
            final_response = "Hemos iniciado el proceso de reprogramacion. Un agente se contactara."

        elif context_data:
            # Respuesta automática
            final_response

>> Inicializando la base de conocimiento...

>> Procesando la consulta: 'Quiero saber si tienen el medicamento para el dolor de cabeza.'
   ->  Clasificacion: Categoria 'otra_consulta'
   -> La consulta es compleja. Activando el RAG.
   -> Buscando contexto relevante...
   -> Generando respuesta con el contexto...

--- Resultado Final del Sistema ---
Consulta del Paciente: Quiero saber si tienen el medicamento para el dolor de cabeza.
Respuesta del Sistema: Disculpe, no tengo una respuesta precisa para su consulta. Por favor, reformule su pregunta.
¿Necesita derivacion humana? No
-----------------------------------

>> Procesando la consulta: 'Como puedo cambiar la hora que tengo con el doctor?'
   ->  Clasificacion: Categoria 'otra_consulta'
   -> La consulta es compleja. Activando el RAG.
   -> Buscando contexto relevante...

--- Resultado Final del Sistema ---
Consulta del Paciente: Como puedo cambiar la hora que tengo con el doctor?
Respuesta del Sistema: Lo siento, no tengo sufici